# Homework 2 - Markov Chain of Letters

This is the answer of Homework 2 given for Bayesian Statistics and Machine Learning. In this answer, functions defined and created by Ali Taylan Cemgil are used. These functions can be found in his github notes. Also, the program runs by the help of the BSML.py file, which involves these functions.

Libraries:

In [ ]:
import numpy as np
import scipy as sc
import time
from BSML import *
import csv

Basic document and functions for conditional probability table

In [ ]:
alphabet = [chr(i+ord('a')) for i in range(26)]
alphabet.append('.')
letter2idx = {c:i for i,c in enumerate(alphabet)}

T = []
with open('transitions.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        T.append(row)

toFloat(stringList): Function for changing the type of float numbers from string to float

In [ ]:
def toFloat(stringList):
    stringList = np.array(stringList,float)

    strList = []
    for i in range(len(stringList)):
        strList.append(normalize(stringList[i][:], axis = 0))
        
    return strList
T = toFloat(T)

dagGen(N): Function to create a Directed Acyclic Graph (DAG) according to a given number that determines its number of nodes. It returns a dictionary including the features of the graph.

In [ ]:
def dagGen(N):
    
    variablenames = ['x' + str(i) for i in range(N)]
    cardinalities = [len(alphabet) for i in range(len(variablenames))]
    parents = {'x0': []}

    for i in range(len(variablenames)):
        if i + 1 == len(variablenames):
            break
        parents.update({'x' + str(i + 1): ['x' + str(i)]})
    states = states_from_cardinalities(variablenames, cardinalities)
    cp_tables = {}
    cp_tables.update({('x0', ): T[letter2idx['.']][:]})
    for i in range(len(variablenames)):
        if i + 1 == len(variablenames):
            break
        x = [T[index][:] for index, j in enumerate(states[variablenames[i]])]
        cp_tables.update({(variablenames[i+1], variablenames[i]) : x})
    
    theta = make_cp_tables(variablenames, cardinalities, cp_tables)

    dagdict = {'variablenames': variablenames,
                  'cardinalities': cardinalities,
                  'parents':parents,
                  'states': states,
                  'cp_tables': cp_tables,
                  'theta': theta}

    return dagdict

indextoletter(dictionary): and statetoletter(statelist): For a given dictionary including nodes and corresponding states, this function returns the corresponding letters being a string. merge(dict1, dict2, variablenames): This function merges two dictionaries which include missing and observed letters according to variable names and returns the complete word as a string.

In [ ]:
def indextoletter(dictionary):
    indexlist = []
    indexlist2 = []
    letterlist = []
    letterlist2 = []
    x = []
    for key in dictionary:
        x.append(key)
    for i in x:
        for j in i:
            indexlist.append(int(j[9:]))
        indexlist2.append(indexlist)
        indexlist = []       
    for i in indexlist2:
        for j in i:
            letterlist.append(alphabet[j])
        letterlist2.append(letterlist)
        letterlist = []
    
    for i in letterlist2:
        letterlist.append("".join(i))
        
    return letterlist

def statetoletter(statelist):
    dictionary = {}
    for i in statelist:
        parts = i.split('_')
        dictionary.update({parts[0]: alphabet[int(parts[2])]})
        
    return dictionary

def merge(dict1, dict2, variablenames):
    letters = []
    for i in variablenames:
        if i in dict1:
            letters.append(dict1[i])
        elif i in  dict2:
            letters.append(dict2[i])
    return "".join(letters)

sample(N, num_of_samples = 1): create word samples according to the given number N corresponding to number of nodes and num_of_samples that correspond to number of samples that is required.

In [ ]:
def sample(N, num_of_samples = 1):
    dagdict = dagGen(N)
    eng = Engine(dagdict['variablenames'], dagdict['parents'],
             dagdict['states'], dagdict['theta'])

    s = sample_states(dagdict['variablenames'], dagdict['states'],
              dagdict['variablenames'], dagdict['parents'],
              dagdict['theta'], num_of_samples)
    return indextoletter(s)

afterobservation(word, parameter = 'randomsample'):
This function takes a word with missing and observed letters, also a parameter that can be either 'randomsample' or 'argmax'. When it takes argmax, it returns a sample of the missing states that have the maximum possibility with the log probability as a list. If it takes 'randomsample', it returns a sample of the missing states according to computed probability table with the log probability as a list.

In [ ]:
def afterobservation(word, parameter = 'randomsample'):
    missinglist = []
    seendict = {}
    word = list(word)
    for index, i in enumerate(word):
        if i == '_' or i == '?':
            missinglist.append('x' + str(index))
        else:
            seendict.update({'x' + str(index): 'x' + str(index) + '_state_' + str(letter2idx[i])})
    
    dagdict = dagGen(len(word))
    eng = Engine(dagdict['variablenames'], dagdict['parents'],
             dagdict['states'], dagdict['theta'], visible_names = missinglist)
    eng.propagate_observation(seendict)
    marg = eng.marginal_table(missinglist)
    
    factor=1.0/sum(marg.values())
    marg_d = {k: v*factor for k, v in marg.items() }
    length = len(marg_d.keys())

    statelist = list(marg_d.keys())
    stateProb = list(marg_d.values())
    length = len(statelist)
    
    if parameter == 'randomsample':
        prob = np.random.choice(list(range(length)), p=list(marg_d.values()))
        missingdict = statetoletter(statelist[prob])
        stateProb = stateProb[prob]
    elif parameter == 'argmax':
        v=list(marg_d.values())
        k=list(marg_d.keys())
        missingdict = statetoletter(k[v.index(max(v))])
        stateProb = max(v)
    
    seendict = statetoletter(list(seendict.values()))
    wordsample = merge(missingdict, seendict, dagdict['variablenames'])
           
    return wordsample, np.log(stateProb)

test(test_strings, parametergiven = 'randomsample'): This function takes strings and works with the function mentioned above. It separates given strings according to '.' to maintain computational efficiency.

In [ ]:
def test(test_strings, parametergiven = 'randomsample'):
    a = []
    logprob = 1
    wordsample = []
    wordlogtuple = []
    if not isinstance(test_strings, list):
        test_strings = [test_strings]
    for i in test_strings:
        a.append(i.split('.'))
    for i in a:
        for index, j in enumerate(i):
            length = len(i)
            if len(j) == 0:
                break
            result  = afterobservation(j, parameter = parametergiven)
            wordsample.append(result[0])
            if result[1] == 0:
                logprob = 1 * logprob
            else:
                logprob = result[1] * logprob
        logprob = logprob / length
        word = '.'.join(wordsample)
        wordlogtuple.append((word, logprob))
        logprob = 1
    
    return wordlogtuple

# Test section of the homework

For the first question. N corresponds to number of nodes and num_of_samples corresponds to the number of sample that is returned.

In [ ]:
print(sample(5, num_of_samples = 4))

For the second question. It takes the test_strings, computes missing letters in each string according to computed probability table and returns the sample with its log probability

In [ ]:
teststrings = []
for i in test_strings:
    teststrings.append(test(i, 'randomsample'))
    time.sleep(25)
print(teststrings)

For the third question. It takes the test_strings, computes missing letters in each string according to argmax and returns the sample with its log probability

In [ ]:
teststrings = []
for i in test_strings:
    teststrings.append(test(i, 'argmax'))
    time.sleep(25)
print(teststrings)